In [1]:
from elexon import ElexonRawClient
from calendar import monthrange
import datetime
import numpy as np
import pandas as pd
from tabulate import tabulate
from tqdm import tqdm
from sys import exit
from tqdm import tqdm
import os
from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.backends.backend_pdf import PdfPages
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Looks Like 2013 onwards looks good.

In [117]:
StartingYear = 2013
EndYear = 2024
Years = tqdm(range(StartingYear,EndYear+1))
DataFrame = pd.DataFrame(columns=["Date","Hour","Starting_Volume(MWh)"])
#
for Year in Years:
    dir_name = f"0.DataSource/NordPool/MarketCrossData/{Year}"
    for file in os.listdir(dir_name): 
        filename = f"{dir_name}/{file}"
        fn = filename.split('_')[1]
        Date = fn.split('.')[0]
        Years.set_description(f'Extracting: {Year} ({file}) ({Date}) ##>>')
        if Year >= 2021:
            data = pd.read_csv(filename,encoding='utf-8') # Encoding is different between old and new files.
        else:
            data = pd.read_csv(filename,encoding='cp1252') # Encoding is different between old and new files, NordPool must have done something to their data acquisition systems.
        data.columns = ['MarketCross','SellOrBuy','Hour','Price','Volume','NotSure']
        data = data.loc[(data["MarketCross"] == "MC") & (data["SellOrBuy"] == "S") ]
        data[["Hour", "Price", "Volume"]] = data[["Hour", "Price","Volume"]].apply(pd.to_numeric)
        #
        for Hour in range (1,25+1): # This starting from 1 and 25 sorts out the daylight saving stuff.
            DataList = []
            hourdata = data.loc[data["Hour"] == Hour]
            MinVolume = hourdata["Volume"].min()
            if abs(MinVolume) > 0:
                DataList.extend([Date,Hour,MinVolume])
                DataFrame.loc[len(DataFrame), DataFrame.columns] = (DataList)
            else:
                continue
DataFrame
DataFrame['Date'] = pd.to_datetime(DataFrame['Date'], format="%Y%m%d")
DataFrame.sort_values(by=['Date','Hour'])
DataFrame.to_csv("0.DataSource/NordPool/MarketCrossData/MC_StartingVolumes2024.csv")

Extracting: 2024 (MC_20240430.csv) (20240430) ##>>: 100%|██████████| 1/1 [00:18<00:00, 18.83s/it]


In [14]:
dir_name = f"0.DataSource/NordPool/MarketCrossData/{2023}"
file = "MC_20231029.csv"
filename = f"{dir_name}/{file}"
#fn = filename.split('_')[1]
#Date = fn.split('.')[0]
#data = pd.read_csv(filename,encoding='utf-8') # Encoding is different between old and new files.
#data = pd.read_csv(filename,encoding='cp1252') # Encoding is different between old and new files, NordPool must have done something to their data acquisition systems.
#data

,# Aggregated Purchase and Sales Curves – N2EX Auction Market,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,#,NaN,NaN,NaN,NaN,NaN
1,# Data type(ST),DeliveryDate,ProcessClock,ProcessDate,NaN,NaN
2,# Data type(BE),Alias,Text,NaN,NaN,NaN
3,# Data type,Code,Alias,Hour,Price(GBP),Volume(MWh/h)
4,# Data type (AL),Number of lines in the file,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
39368,MC,P,25,2000.01,8088.2,NaN
39369,MC,P,25,2000.02,8083.2,NaN
39370,MC,P,25,3486.00,8083.2,NaN
39371,MC,P,25,3486.00,0.0,NaN
